# ShuffleNetV1

首先需要了解分组卷积的概念

## 基本单元如下

![unit](shuffnetv1_unit.png)

## 结构如下

repeat为重复次数,g列对应的输入,表示group取对应值的时候,该层输出可能会有变化.而对于瓶颈层，将通道设为每个ShuffleNet单元输出通道的1/4

![unit](shuffnetv1_stru.png)

In [7]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
import os
print(torch.__version__)

1.10.1


In [24]:

class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, stride, groups):
        super().__init__()
        # 瓶颈层输出为每个单元输出的1/4
        mid_channles = int(out_channels/4)
        # 作者提到不在stage2的第一个pointwise层使用组卷积,因为输入channel数量太少,只有24
        # 所以第一个stage特殊处理
        if in_channels <=24:
            self.groups = 1
        else:
            self.groups = groups
        self.stride = stride
        # 分组1x1卷积,特征图深度变为本单元输出的1/4
        self.conv1 = nn.Sequential(
            # 第一个stage的第一个1x1可能会不用组卷积,所以特殊处理下
            nn.Conv2d(in_channels, mid_channles, 1, groups=self.groups, bias=False),
            nn.BatchNorm2d(mid_channles),
            nn.ReLU(inplace=True)
        )
        # 跨距为2时,缩小特征图wh,不改变深度,后接bn无relu
        self.conv2 = nn.Sequential(
            nn.Conv2d(mid_channles, mid_channles, 3, stride=stride, padding=1, groups=mid_channles, bias=False),
            nn.BatchNorm2d(mid_channles)
        )
        # 分组1x1卷积,不改变尺寸,但是改变深度为输出深度,后接bn无relu
        self.conv3 = nn.Sequential(
            nn.Conv2d(mid_channles, out_channels, 1, groups=groups, bias=False),
            nn.BatchNorm2d(out_channels)
        )
        if(self.stride==2):
            # 特征图尺寸减为一半,和跨距为2的效果相同
            self.shortcut = nn.Sequential(nn.AvgPool2d(3, stride=2, padding=1))

    def forward(self, x):
        out = self.conv1(x)
        # shuffle单元只对第一个1x1分组卷积之后进行shuffle,作者说第二个1x1卷积之后再做shuffle效果一般
        out = Bottleneck.shuffle(out, self.groups)
        out = self.conv2(out)
        out = self.conv3(out)
        if self.stride == 2:
            # 堆叠后,输出深度为out_channel+in_channel,外层需注意
            res = self.shortcut(x)
            out = F.relu(torch.cat([out, res], 1))
        else:
            # 都是对最后的输出加上relu
            out = F.relu(out+x)
        return out
    @staticmethod
    def shuffle(x, groups):
        N, C, H, W = x.size()
        out = x.view(N, groups, C // groups, H, W).permute(0, 2, 1, 3, 4).contiguous().view(N, C, H, W)
        return out
    
class ShuffleNet(nn.Module):
    channel_num=(
        # 有关各种group对应每层的输出参数
        # group的取值为1,2,3,4,8
        (144,288,576),
        (200,400,800),
        (240,480,960),
        (272,544,1088),
        (),
        (),
        (),
        (384,768,1536),
    )
    groups_area=(1,2,3,4,8)
    def __init__(self, groups, first_channel=3,class_num=10):
        super().__init__()
        self.class_num=class_num
        if not groups in self.groups_area:
            raise ValueError("groups value %s error"%groups)
        self.groups=groups
        self.channel_stru=self.channel_num[self.groups]
        # in 224x224x3
        self.conv1 = nn.Sequential(
            nn.Conv2d(first_channel, 24, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(24),
            nn.ReLU(inplace=True)
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # in 56x56xx24
        self.stage2 = self.make_layers(24, self.channel_stru[0], repeat_times=4, strides=2, groups=groups)
        self.stage3 = self.make_layers(self.channel_stru[0], self.channel_stru[1], repeat_times=8, strides=2, groups=groups)
        self.stage4 = self.make_layers(self.channel_stru[1], self.channel_stru[2], repeat_times=4, strides=2, groups=groups)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(self.channel_stru[2],self.class_num)
    def make_layers(self, input_channels, output_channels, repeat_times, strides, groups):
        layers = []
        if strides!=2:
            raise ValueError("first head conv strides must 2",strides)
        # 第一层跨距为2,肯定会进行堆叠,所以输出层输写成 实际输出-输入层数
        layers.append(Bottleneck(input_channels, output_channels - input_channels, strides, groups))
        input_channels = output_channels
        for i in range(repeat_times - 1):
            Bottleneck(input_channels, output_channels, 1, groups)
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.avgpool(x)
        # 这之后还是ncwh四维的,不过是(n,1024,1,1)这种
        # flatten(1),从第1个维度开始到一直展平到最后一个维度
        x = x.flatten(1)
        x = self.fc(x)
        return x

In [26]:
net=ShuffleNet(groups=2)
data=torch.randn(4,3,224,224)
out=net(data)
out.shape

torch.Size([4, 10])